# Camera Calibration

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import inv
import cv2 

## Preparing the data

In [2]:
data = pd.read_csv('data.csv')
Q = []
p = []
x_map = lambda xo, yo, zo, xi : np.array([xo, yo, zo, 1, 0, 0, 0, 0, -(xi*xo), -(xi*yo), -(xi*zo)])
y_map = lambda xo, yo, zo, yi : np.array([0, 0, 0, 0, xo, yo,zo, 1, -(yi*xo), -(yi*yo), -(yi*zo)])

for i in data.index:
    xo = data.loc[i,'xo']
    yo = data.loc[i,'yo']
    zo = data.loc[i,'zo']
    xi = data.loc[i,'xi']
    yi = data.loc[i,'yi']
    
    Q_temp_row1 = x_map(xo, yo, zo, xi)
    Q_temp_row2 = y_map(xo, yo, zo, yi)
    
    Q.append(Q_temp_row1)
    Q.append(Q_temp_row2)
    p.append(xi)
    p.append(yi)
    
Q = np.array(Q)
p = np.array(p)

## Solving for $a$

In [3]:
a = np.matmul(inv(np.matmul(Q.T, Q)), np.matmul(Q.T, p))
a11 = a[0]
a12 = a[1]
a13 = a[2]
a14 = a[3]
a21 = a[4]
a22 = a[5]
a23 = a[6]
a24 = a[7]
a31 = a[8]
a32 = a[9]
a33 = a[10]
a34 = 1

## Testing the accuracy of $a$

In [4]:
img = cv2.imread("Tsai grid.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
x_pixels = np.arange(0, img.shape[1], 1)
y_pixels = np.arange(0, img.shape[0], 1)
X, Y = np.meshgrid(x_pixels, y_pixels)

def mark_points(coords):
    A = np.zeros(X.shape)
    
    for i in coords:
        R = np.sqrt((X - i[0])**2  + (Y - i[1])**2)
        A[10 > R] = 1

    return A

In [5]:
test = pd.read_csv("Data.csv")
predicted_pos = []

for i in test.index:
    xo = test.loc[i,'xo']
    yo = test.loc[i,'yo']
    zo = test.loc[i,'zo']
    
    xi = (a11*xo + a12*yo + a13*zo + a14) / (a31*xo + a32*yo + a33*zo + a34)
    yi = (a21*xo + a22*yo + a23*zo + a24) / (a31*xo + a32*yo + a33*zo + a34)

    predicted_pos.append([int(xi), int(yi)])
    
P_test = np.array(predicted_pos)
mask = mark_points(P_test)
img[mask == 1] = [255, 0, 0]

In [6]:
predicted_pos

[[236, 495],
 [156, 545],
 [214, 467],
 [190, 434],
 [143, 456],
 [246, 387],
 [223, 325],
 [147, 285],
 [111, 442],
 [127, 350],
 [299, 493],
 [378, 540],
 [356, 506],
 [316, 464],
 [383, 450],
 [337, 407],
 [296, 370],
 [296, 323],
 [389, 343],
 [278, 342]]